Window size - number of  of past values we are using  to predict  the  future value

<h3>Loading the libraries <h3>

In [1]:
import tensorflow as tf
import pickle

In [2]:
dataset =tf.data.Dataset.range(10)  
#for val in dataset:
    #print(val)  
    # type of this val is tensor object, in our case it's scalar
    #print(val.numpy())
    # turning tensor object into numpy array

dataset=dataset.window(5,shift=1, drop_remainder=True)
# now this data  is a window dataset
# windowed dataset ->
'''[  Dataset([0, 1, 2, 3, 4]),
      Dataset([1, 2, 3, 4, 5]),
      Dataset([2, 3, 4, 5, 6]),
      Dataset([3, 4, 5, 6, 7]),
      Dataset([4, 5, 6, 7, 8]),
      Dataset([5, 6, 7, 8, 9])
]
'''
dataset = dataset.flat_map(lambda window: window.batch(5))
# here we are taking each window,
# batching  each windows elements into a tensor object
#Dataset[1,2,3,4,5] -> tensor flow object(1,2,3,4,5)
for val in dataset:
    print(val.numpy())
    #val.numpy() will turn the tensor flow object into numpy array
    





[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


2024-05-18 17:14:49.494285: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3> Creating   the dataset <h3>

In [3]:
dataset = dataset.map(lambda window: (window[:-1],window[-1]))
for x, y in dataset:
    print(x.numpy(),y.numpy())
 # here we are mapping  window (type tensor object) element into a tuple of 
 #(first 4 elements of the window, last element of  the window)   

[0 1 2 3] 4
[1 2 3 4] 5
[2 3 4 5] 6
[3 4 5 6] 7
[4 5 6 7] 8
[5 6 7 8] 9


2024-05-18 17:14:49.560713: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3> Shuffling <h3>


In [4]:
dataset =  dataset.shuffle(buffer_size=10)
for x, y in dataset:
    print(x.numpy(),y.numpy())


[5 6 7 8] 9
[3 4 5 6] 7
[0 1 2 3] 4
[1 2 3 4] 5
[4 5 6 7] 8
[2 3 4 5] 6


2024-05-18 17:14:49.591570: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3> Batching <h3>

In [5]:
dataset= dataset.batch(2).prefetch(1)
for x,y in dataset:
    print("x="  , x.numpy())
    print("y=" ,y.numpy())

x= [[4 5 6 7]
 [2 3 4 5]]
y= [8 6]
x= [[3 4 5 6]
 [0 1 2 3]]
y= [7 4]
x= [[5 6 7 8]
 [1 2 3 4]]
y= [9 5]


2024-05-18 17:14:49.622509: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3> Combining all together into a function <h3>

In [6]:
def windowed_dataset (series,window_size,batch_size ,shuffle_buffer):
    #creating tensor flow object from the series
    dataset =  tf.data.Dataset.from_tensor_slices(series)
    #creating dataset of windows
    dataset = dataset.window(window_size+1,shift=1,drop_remainder=True)
    #turning the dataset of windows into dataset  of numpy arrays
    dataset.flat_map(lambda window : window.batch(window_size+1))
    #shuffling and mapping the each  window dataset into a tuple of (features,label))   
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window:(window[:1],window[-1]))
    #creating batches
    dataset=dataset.batch(batch_size).prefetch(1)
    return dataset
    

<h3> Training a single layer neural network on the time series <h3>

In [9]:


# Load the serialized DataFrame from the file
with open('product_df.pkl', 'rb') as f:
    product_df = pickle.load(f)

# Now you can use the product_df variable in this notebook
print(product_df)


            Sales
2010-05-31     40
2010-06-30   3375
2010-07-31   3633
2010-08-31   1926
2010-09-30    801
...           ...
2022-08-31   3321
2022-09-30   1069
2022-10-31    870
2022-11-30    359
2022-12-31    298

[152 rows x 1 columns]
